In [6]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)

In [8]:
conn = sqlite3.connect('oil_detection.db')
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

for table in tables['name']:
    print(f"\nTabela: {table}")
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 2;", conn)
    display(df)

conn.close()


Tabela: empresas


,empresa_id,empresa_nome,created_at
0,DOS0724,Data Overseas,2025-06-14 21:27:09



Tabela: coletores


,coletor_id,empresa_id,coletor_descricao,coletor_localizacao,status,created_at
0,APS005,DOS0724,Camera Costeiro - Santos,"-23.9608, -46.3336",ativo,2025-06-14 21:27:09



Tabela: deteccoes


...

,id,deteccao_id,object_label,bbox_x1,bbox_y1,bbox_x2,bbox_y2,created_at
0,1,1,furniture,0.32,282.0,243.519989,479.279999,2025-06-14 21:27:22



Tabela: grounding_dino_results


,id,deteccao_id,oil_spill_detected,max_confidence,total_detections,image_width,image_height,created_at
0,1,1,1,0.641157,1,640,480,2025-06-14 21:27:22



Tabela: grounding_detections


,id,grounding_result_id,phrase,confidence,bbox_x1,bbox_y1,bbox_x2,bbox_y2,created_at
0,1,1,oil spill [SEP],0.641157,0.189048,0.814803,0.376693,0.365242,2025-06-14 21:27:22


In [9]:
# Conectar ao banco de dados novamente
conn = sqlite3.connect('oil_detection.db')

# Consulta SQL para juntar todas as tabelas relacionadas
query = """
SELECT 
    d.*,
    e.empresa_nome,
    c.coletor_descricao, c.coletor_localizacao, c.status as coletor_status,
    fc.caption_text, fc.caption_type,
    fdr.region_text, fdr.bbox_x1 as region_x1, fdr.bbox_y1 as region_y1, fdr.bbox_x2 as region_x2, fdr.bbox_y2 as region_y2,
    fo.object_label, fo.bbox_x1 as object_x1, fo.bbox_y1 as object_y1, fo.bbox_x2 as object_x2, fo.bbox_y2 as object_y2,
    gdr.oil_spill_detected, gdr.max_confidence, gdr.total_detections, gdr.image_width, gdr.image_height,
    gd.phrase, gd.confidence as grounding_confidence, 
    gd.bbox_x1 as grounding_x1, gd.bbox_y1 as grounding_y1, gd.bbox_x2 as grounding_x2, gd.bbox_y2 as grounding_y2
FROM 
    deteccoes d
LEFT JOIN empresas e ON d.empresa_id = e.empresa_id
LEFT JOIN coletores c ON d.coletor_id = c.coletor_id
LEFT JOIN florence_captions fc ON d.id = fc.deteccao_id
LEFT JOIN florence_dense_regions fdr ON d.id = fdr.deteccao_id
LEFT JOIN florence_objects fo ON d.id = fo.deteccao_id
LEFT JOIN grounding_dino_results gdr ON d.id = gdr.deteccao_id
LEFT JOIN grounding_detections gd ON gdr.id = gd.grounding_result_id
"""

# Criar o DataFrame com todos os dados
df = pd.read_sql_query(query, conn)

# Fechar a conexão
conn.close()

# Mostrar as primeiras linhas do DataFrame
df


,id,empresa_id,coletor_id,timestamp_coleta,timestamp_processamento,modelo_usado,imagem_original_path,imagem_processada_path,confidence_yolo,objects_detected_yolo,empresa_nome,coletor_descricao,coletor_localizacao,coletor_status,caption_text,caption_type,region_text,region_x1,region_y1,region_x2,region_y2,object_label,object_x1,object_y1,object_x2,object_y2,oil_spill_detected,max_confidence,total_detections,image_width,image_height,phrase,grounding_confidence,grounding_x1,grounding_y1,grounding_x2,grounding_y2
0,1,DOS0724,APS005,2025-06-14T18:27:00.918292,2025-06-14 21:27:22,oil_spill,Archived\DOS0724\APS005\oil_spill\raw_image\RA...,None,0.7,1,Data Overseas,Camera Costeiro - Santos,"-23.9608, -46.3336",ativo,A shadow of a person sitting in a chair.,caption,chair,0.32,282.0,243.519989,479.279999,furniture,0.32,282.0,243.519989,479.279999,1,0.641157,1,640,480,oil spill [SEP],0.641157,0.189048,0.814803,0.376693,0.365242
1,1,DOS0724,APS005,2025-06-14T18:27:00.918292,2025-06-14 21:27:22,oil_spill,Archived\DOS0724\APS005\oil_spill\raw_image\RA...,None,0.7,1,Data Overseas,Camera Costeiro - Santos,"-23.9608, -46.3336",ativo,In this image we can see a person's shadow on ...,detailed_caption,chair,0.32,282.0,243.519989,479.279999,furniture,0.32,282.0,243.519989,479.279999,1,0.641157,1,640,480,oil spill [SEP],0.641157,0.189048,0.814803,0.376693,0.365242
2,1,DOS0724,APS005,2025-06-14T18:27:00.918292,2025-06-14 21:27:22,oil_spill,Archived\DOS0724\APS005\oil_spill\raw_image\RA...,None,0.7,1,Data Overseas,Camera Costeiro - Santos,"-23.9608, -46.3336",ativo,A shadow of a person's face can be seen on a w...,more_detailed_caption,chair,0.32,282.0,243.519989,479.279999,furniture,0.32,282.0,243.519989,479.279999,1,0.641157,1,640,480,oil spill [SEP],0.641157,0.189048,0.814803,0.376693,0.365242
